## Imports

In [1]:
from sklearn.pipeline import Pipeline
import bootcampviztools as bt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import Toolbox_ML as tb
import lightgbm as lgb

from collections import Counter
from lightgbm import LGBMClassifier, LGBMRegressor
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, SelectFromModel, RFE, SequentialFeatureSelector
from sklearn.metrics import mutual_info_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, SelectFromModel, f_regression
from catboost import CatBoostRegressor
import pickle
import warnings
warnings.filterwarnings("ignore")
import re
from sklearn.compose import make_column_selector as selector

## Limpieza del train

In [2]:
df = pd.read_csv("./data/train.csv")

#### Funciones de limpieza

In [3]:
# Función para convertir las capacidades a GBs y hacerlas numéricas
def convert_capacity(capacity_str):
    if 'tb' in capacity_str:
        return float(capacity_str.replace('tb', '').strip()) * 1000
    elif 'gb' in capacity_str:
        return float(capacity_str.replace('gb', '').strip())
    else:
        return 0

# Función para descomponer cada entrada en los componentes especificados
def decompose_info(row):
    # Buscar la capacidad de almacenamiento
    capacities = re.findall(r'\d+(?:\.\d+)?[tg]b', row)
    total_capacity = sum(convert_capacity(c) for c in capacities)
    
    # Buscar el tipo de almacenamiento
    if 'ssd' in row and 'hdd' in row:
        storage_type = 'ssd/hdd'
    elif 'SSD' in row and 'hybrid' in row:
        storage_type = 'SSD/Hybrid'
    elif 'hdd' in row and 'flash storage' in row:
        storage_type = 'hdd/flash storage'
    elif 'ssd' in row:
        storage_type = 'ssd'
    elif 'hdd' in row:
        storage_type = 'hdd'
    elif 'flash storage' in row:
        storage_type = 'flash storage'
    elif 'hybrid' in row:
        storage_type = 'hybrid'
    
    # Buscar la combinación de almacenamiento
    if '+' in row:
        multiple_storage = True
    else:
        multiple_storage = False
    
    return pd.Series([total_capacity, storage_type, multiple_storage])

def split_info(row):
    # Extraer marca (Intel o AMD)
    if 'intel' in row:
        brand = 'intel'
    elif 'amd' in row:
        brand = 'amd'
    else:
        brand = None
    
    # Extraer frecuencia (con Ghz)
    ghz = re.search(r'\d+(\.\d+)?ghz', row)
    ghz = ghz.group() if ghz else None
    
    # Extraer el resto
    rest = row
    if brand:
        rest = rest.replace(brand, '').strip()
    if ghz:
        rest = rest.replace(ghz, '').strip()
    
    return pd.Series([brand, ghz, rest])

def split_info_2(row):
    # Extraer marca (Intel Nvidia o AMD)
    if 'intel' in row:
        brand = 'intel'
    elif 'amd' in row:
        brand = 'amd'
    elif 'nvidia' in row:
        brand = 'nvidia'
    else:
        brand = None
    
    # Extraer el resto
    rest = row
    if brand:
        rest = rest.replace(brand, '').strip()
    
    return pd.Series([brand, rest])

def decompose_info_2(row):
    # Buscar resolución en el formato "????x????"
    resolution = re.search(r'\b\d{3,4}x\d{3,4}\b', row)
    resolution = resolution.group() if resolution else None

    # Determinar el tipo de panel
    panel_type = 'ips Panel' in row
    
    # Determinar la calidad de imagen
    if '4k ultra hd' in row:
        quality = '4k ultra hd'
    elif 'quad hd+' in row:
        quality = 'quad hd+'
    elif 'full hd' in row:
        quality = 'full hd'
    elif 'hd' in row:
        quality = 'hd'
    else:
        quality = None
    
    # Determinar si es Touchscreen
    is_touchscreen = 'touchscreen' in row
    
    return pd.Series([resolution, panel_type, quality, is_touchscreen])

#### Target x e y

In [4]:
target = 'Price_euros'
X = df.drop(target, axis = 1)
y = df[target]

#### Creación de nuevas columnas

In [5]:
# Hago todos los valores minúsculas
X = X.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Descompongo Memory, Cpu, Gpu, ScreenResolution y Product
X[['Capacity', 'Storage Type', 'Multiple Storage']] = X['Memory'].apply(decompose_info)
X[['Marca CPU', 'GHz CPU', 'Modelo CPU']] = X['Cpu'].apply(split_info)
X[['Marca GPU', 'Modelo GPU']] = X['Gpu'].apply(split_info_2)
X[['Modelo CPU', 'Submodelo CPU']] = X['Modelo CPU'].str.rsplit(n=1, expand=True)
X[['Modelo GPU', 'Submodelo GPU']] = X['Modelo GPU'].str.rsplit(n=1, expand=True)
X[['Resolution', 'Panel Type', 'Quality', 'Touchscreen']] = X['ScreenResolution'].apply(decompose_info_2)
X[['Modelo', 'Submodelo']] = X['Product'].str.extract(r'(\w+)\s*(.*)')
X['Submodelo'] = X['Submodelo'].str.replace('-', ' ').str.split('(').str[0].str.strip()

# Quito GHz, GB y Kg de estas columnas
X['GHz CPU'] = X['GHz CPU'].str.replace('ghz','',regex=False)
X['GHz CPU'] = X['GHz CPU'].astype(float)
X.Weight = X.Weight.str.replace('kg','',regex=False)
X.Weight = X.Weight.astype(float)
X.Ram = X.Ram.str.replace('gb','',regex=False)
X.Ram = X.Ram.astype(float)

# Los valores vacíos los transformo en None
X.replace(r'^\s*$', None, regex=True, inplace=True)

#### Drop de columnas antiguas

In [7]:
X = X.drop(columns=['laptop_ID', 'id', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'Product'])

In [8]:
X

,Company,TypeName,Inches,Ram,OpSys,Weight,Capacity,Storage Type,Multiple Storage,Marca CPU,...,Marca GPU,Modelo GPU,Resolution,Panel Type,Quality,Touchscreen,Modelo,Submodelo,Submodelo CPU,Submodelo GPU
0,dell,notebook,15.6,8.0,windows 10,2.36,1000.0,hdd,False,intel,...,amd,radeon r7,1366x768,False,None,False,inspiron,5567,7500u,m445
1,asus,notebook,15.6,4.0,windows 10,2.00,1000.0,hdd,False,intel,...,intel,hd graphics,1366x768,False,None,False,x541na,None,n4200,505
2,toshiba,notebook,13.3,8.0,windows 7,1.20,256.0,ssd,False,intel,...,intel,hd graphics,1920x1080,False,full hd,False,portege,z30 c 1cw,6200u,520
3,dell,gaming,15.6,16.0,windows 10,4.42,1256.0,ssd/hdd,True,intel,...,nvidia,geforce gtx,3840x2160,False,4k ultra hd,False,alienware,17,7700hq,1070
4,dell,ultrabook,12.5,8.0,windows 7,1.26,256.0,ssd,False,intel,...,intel,hd graphics,1920x1080,False,full hd,True,latitude,e7270,6300u,520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,acer,2 in 1 convertible,11.6,4.0,chrome os,1.25,32.0,flash storage,False,intel,...,intel,hd graphics,1366x768,False,None,True,chromebook,c738t c2ej,n3060,400
908,toshiba,notebook,13.3,4.0,windows 10,1.20,128.0,ssd,False,intel,...,intel,hd graphics,1920x1080,False,full hd,False,portege,z30 c 16h,6200u,520
909,acer,notebook,15.6,8.0,windows 10,2.40,1000.0,hdd,False,amd,...,amd,radeon,1366x768,False,None,False,aspire,es1 523,7410,r5
910,dell,ultrabook,14.0,8.0,windows 10,1.36,256.0,ssd,False,intel,...,intel,hd graphics,1920x1080,False,full hd,False,latitude,7480,7600u,620


## Creación de la pipeline

In [9]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar las características numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Crear transformadores para las características numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar los transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


#### Pruebas con RF

In [24]:
# Crear la pipeline completa con un modelo de regresión
pipeline_1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Definir la cuadrícula de hiperparámetros para el RandomForestRegressor
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],       # Número de árboles en el bosque
    'regressor__max_depth': [None, 10, 20, 30],           # Profundidad máxima del árbol
    'regressor__min_samples_split': [2, 5, 10],           # Número mínimo de muestras necesarias para dividir un nodo
    'regressor__min_samples_leaf': [1, 2, 4],             # Número mínimo de muestras necesarias en un nodo hoja
    'regressor__max_features': ['auto', 'sqrt', 'log2'],  # Número de características a considerar para la mejor división
    'regressor__bootstrap': [True, False]
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipeline_1, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_percentage_error')

# Ajustar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mejor conjunto de hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Realizar predicciones en el conjunto de prueba con el mejor modelo encontrado
best_model_1 = grid_search.best_estimator_
y_pred_1 = best_model_1.predict(X_test)

# Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred_1)
print("Error absoluto medio del modelo:", mae)

Mejores hiperparámetros: {'regressor__bootstrap': False, 'regressor__max_depth': None, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Error absoluto medio del modelo: 175.7974606557377


#### Pruebas con LightGBM

In [30]:
# Crear la pipeline completa con un modelo de regresión
pipeline_2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(n_estimators=100, random_state=42))
])

# Definir la cuadrícula de hiperparámetros para el RandomForestRegressor
param_grid = {
    'regressor__num_leaves': [31, 40, 50],                 # Número máximo de hojas en un árbol
    'regressor__max_depth': [-1, 10, 20, 30],              # Profundidad máxima del árbol
    'regressor__learning_rate': [0.01, 0.1, 0.2],          # Tasa de aprendizaje
    'regressor__n_estimators': [100, 200, 300],            # Número de árboles
    'regressor__min_child_samples': [20, 30, 40],          # Número mínimo de muestras en una hoja
    'regressor__subsample': [0.6, 0.8, 1.0],               # Fracción de datos para entrenar cada árbol
    'regressor__colsample_bytree': [0.6, 0.8, 1.0]         # Fracción de características para entrenar cada árbol
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipeline_2, param_grid, cv=5, n_jobs = -1, scoring='neg_mean_absolute_percentage_error')

# Ajustar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mejor conjunto de hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Realizar predicciones en el conjunto de prueba con el mejor modelo encontrado
best_model_2 = grid_search.best_estimator_
y_pred_2 = best_model_2.predict(X_test)

# Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred_2)
print("Error absoluto medio del modelo:", mae)

#### Pruebas con XGBoost

In [13]:
# Crear la pipeline completa con un modelo de regresión
pipeline_3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        alpha=0.005,                      # Parámetro de regularización L1 (alpha)
        reg_lambda=0.005,                 # Parámetro de regularización L2 (lambda)
        gamma=0.005                       # Parámetro de regularización en la reducción mínima de la pérdida (gamma)
    ))
])

# Definir la cuadrícula de hiperparámetros para el XGBoost
param_grid = {
    'regressor__n_estimators': [725, 735, 745],
    'regressor__max_depth': [7.5, 8, 8.5],
    'regressor__learning_rate': [0.05, 0.1, 0.2],
    'regressor__subsample': [0.6, 0.8, 1],
    'regressor__colsample_bytree': [0.2, 0.4, 0.6],
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipeline_3, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_percentage_error')

# Ajustar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mejor conjunto de hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Realizar predicciones en el conjunto de prueba con el mejor modelo encontrado
best_model_3 = grid_search.best_estimator_
y_pred_3 = best_model_3.predict(X_test)

# Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred_3)
print("Error absoluto medio del modelo:", mae)

Mejores hiperparámetros: {'regressor__colsample_bytree': 0.2, 'regressor__learning_rate': 0.05, 'regressor__max_depth': 8, 'regressor__n_estimators': 735, 'regressor__subsample': 0.6}
Error absoluto medio del modelo: 127.55910199941833


- Modelo 1:

param_grid = {
    'regressor__n_estimators': [300, 600, 900],
    'regressor__max_depth': [7, 9, 11],
    'regressor__learning_rate': [0.1, 0.2, 0.4, 0.8],
    'regressor__subsample': [0.1, 0.2, 0.4, 0.8],
    'regressor__colsample_bytree': [0.2, 0.4, 0.8],
}

neg_mean_absolute_percentage_error

Mejores hiperparámetros: {'regressor__colsample_bytree': 0.4, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 9, 'regressor__n_estimators': 600, 'regressor__subsample': 0.8}
Error absoluto medio del modelo: 135.14566630712622

- Modelo 2:

param_grid = {
    'regressor__n_estimators': [725, 735, 745],
    'regressor__max_depth': [7.5, 8, 8.5],
    'regressor__learning_rate': [0.05, 0.1, 0.2],
    'regressor__subsample': [0.6, 0.8, 1],
    'regressor__colsample_bytree': [0.2, 0.4, 0.6],
}

Mejores hiperparámetros: {'regressor__colsample_bytree': 0.2, 'regressor__learning_rate': 0.05, 'regressor__max_depth': 8, 'regressor__n_estimators': 735, 'regressor__subsample': 0.6}
Error absoluto medio del modelo: 127.55910199941833

In [142]:
# Para mirar la matriz de importancias

pipeline_3.fit(X_train, y_train)
model = pipeline_3.named_steps['regressor']
importancias = model.feature_importances_
column_names = preprocessor.transformers_[0][1].named_steps['scaler'].get_feature_names_out(numeric_features).tolist()
column_names.extend(preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features).tolist())

importancias_df = pd.DataFrame({'Características': column_names, 'Importancia': importancias})
importancias_df = importancias_df.sort_values(by='Importancia', ascending=False)


#### Pruebas con CatBoost

In [67]:
# Crear la pipeline completa con un modelo de regresión
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(random_state=42, silent= True))
])

# Definir la cuadrícula de hiperparámetros para el CatBoost
param_grid = {
    'regressor__iterations': [300, 600, 900],
    'regressor__depth': [7, 9, 11],
    'regressor__learning_rate': [0.1, 0.2, 0.4, 0.8],
    'regressor__subsample': [0.1, 0.2, 0.4, 0.8],
    'regressor__colsample_bylevel': [0.2, 0.4, 0.8]
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_percentage_error')

# Ajustar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mejor conjunto de hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Realizar predicciones en el conjunto de prueba con el mejor modelo encontrado
best_model_4 = grid_search.best_estimator_
y_pred_4 = best_model_4.predict(X_test)

# Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred_4)
print("Error absoluto medio del modelo:", mae)

Mejores hiperparámetros: {'regressor__colsample_bylevel': 0.4, 'regressor__depth': 7, 'regressor__iterations': 900, 'regressor__learning_rate': 0.1, 'regressor__subsample': 0.2}
Error absoluto medio del modelo: 142.3231006814806


- Modelo 1:

param_grid = {
    'regressor__iterations': [300, 600, 900],
    'regressor__depth': [7, 9, 11],
    'regressor__learning_rate': [0.1, 0.2, 0.4, 0.8],
    'regressor__subsample': [0.1, 0.2, 0.4, 0.8],
    'regressor__colsample_bylevel': [0.2, 0.4, 0.8]
}

Mejores hiperparámetros: {'regressor__colsample_bylevel': 0.4, 'regressor__depth': 7, 'regressor__iterations': 900, 'regressor__learning_rate': 0.1, 'regressor__subsample': 0.2}
Error absoluto medio del modelo: 142.3231006814806

#### Guardar la pipeline

In [14]:
import pickle

pipeline_entrenada = pipeline_3

with open('mi_pipeline_entrenada.pkl', 'wb') as archivo:
    pickle.dump(pipeline_entrenada, archivo)

------

## Limpieza del test

In [15]:
test = pd.read_csv("./data/test.csv")

In [16]:
test = test.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [17]:
test.Weight = test.Weight.str.replace('kg','',regex=False)
test.Weight = test.Weight.astype(float)
test.Ram = test.Ram.str.replace('gb','',regex=False)
test.Ram = test.Ram.astype(float)

In [18]:
test[['Capacity', 'Storage Type', 'Multiple Storage']] = test['Memory'].apply(decompose_info)
test[['Marca CPU', 'GHz CPU', 'Modelo CPU']] = test['Cpu'].apply(split_info)
test[['Marca GPU', 'Modelo GPU']] = test['Gpu'].apply(split_info_2)
test[['Resolution', 'Panel Type', 'Quality', 'Touchscreen']] = test['ScreenResolution'].apply(decompose_info_2)
test[['Modelo CPU', 'Submodelo CPU']] = test['Modelo CPU'].str.rsplit(n=1, expand=True)
test[['Modelo GPU', 'Submodelo GPU']] = test['Modelo GPU'].str.rsplit(n=1, expand=True)
test['GHz CPU'] = test['GHz CPU'].str.replace('ghz','',regex=False)
test['GHz CPU'] = test['GHz CPU'].astype(float)
test[['Modelo', 'Submodelo']] = test['Product'].str.extract(r'(\w+)\s*(.*)')
test['Submodelo'] = test['Submodelo'].str.replace('-', ' ').str.split('(').str[0].str.strip()
test.replace(r'^\s*$', None, regex=True, inplace=True)
test_copia = test.copy()
test = test.drop(columns=['laptop_ID', 'id', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'Product'])

- Modelos que he ido guardando

In [103]:
modelo_135_xgb = best_model_3

In [121]:
modelo_144_cat = best_model_4

In [136]:
modelo_140_xgb = best_model_3

In [162]:
modelo_134_xgb = best_model_3

In [182]:
modelo_129_xgb = best_model_3

## Predicciones según modelo

In [26]:
predicciones = best_model_1.predict(test) # RF

In [20]:
predicciones = best_model_2.predict(test) # LGBM

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [19]:
predicciones = best_model_3.predict(test) # XGBoost

In [131]:
predicciones = modelo_144_cat.predict(test) # CatBoost

In [20]:
# Mostrar las predicciones
print("Predicciones:")
print(predicciones)

Predicciones:
[2066.0745   650.08734  376.75833 1455.0317   965.32196 1206.2781
 3076.0723   562.3106  2742.5908  2134.7017  1590.4265   514.09143
  353.12634  544.4695   418.68707 1316.9823   524.1895  1852.2615
 1619.1824  1635.4423  1782.8779  1255.6631  1545.029   1402.8821
 2034.3118   478.23254  974.61237  382.4973  1206.6698   538.29987
  978.9086  1224.7634  1934.8323   387.99252  622.7391  1681.8741
  820.62836  830.71313 2990.2832   537.05365  595.8353   878.5401
 1214.5981  2034.8364  1118.8264  1785.5459   589.90173 3501.9104
  262.80768 1221.3885  1057.8702  1031.6669   995.9561   336.6446
 1058.1384   711.6742   744.1978   229.74516 1324.6218  1068.0214
 1150.1045  1068.6111   628.5409  1150.2555  1311.0963  1443.2578
  678.8779  2513.814    523.9263  2104.4514   937.6862  2766.5115
 1756.3304  1002.5143  1444.0303  1188.4753   840.8143   486.71854
 1080.9127  1166.7083   299.27222 1547.9215  1656.9849   914.74866
 1208.7214  1470.5593  2213.8923   497.01227 1889.5885   5

#### Crear df y pasarlo a formato csv

In [21]:
df_final_xgb_mae_127 = pd.DataFrame(data={'id': test_copia['id'], 'Price_euros': predicciones})

In [22]:
df_final_xgb_mae_127.to_csv('hola_127.csv', header=True, index= False)

-----